### 0. Install Julia and POMDP.jl

Please first install Julia and POMDP.jl (https://github.com/JuliaPOMDP/POMDPs.jl).


### 1. Include Julia packages
Several POMDP julia packages such as "POMDPTools, POMDPModels" are already included in planner.jl.

If there is an error about uninstalled packages, please install them with "pkg.add()".

In [ ]:
include("./ContinuousPOMCGS/Planner.jl")
# --- Light Dark ---
pomdp = LightDark1D()

### 2. Define parameters

In [ ]:
b0 = initialstate(pomdp) # Get POMDP's initial belief
nb_process_samples = 50000
state_grid_distance = [1.0, 1.0]
max_b_gap = 0.1
nb_particles_b0 = nb_process_samples
c = 2.0
max_planning_secs = 3600
nb_abstract_obs = 10
epsilon = 0.01
particles_b0, init_map_b0_d2c_states, dict_weighted_b0, action_space = InitElemPOMDP(pomdp, b0, state_grid_distance, nb_particles_b0)
rmin, a_rmin = FindRLower(pomdp, b0, action_space)

### 3. Prepare $V_{mdp}$

Here we use a Q-learning algorithm to compute $V_{MDP}$, one can also use other MDP solvers.

In [ ]:
Q_table = Dict{Any, Dict{Int64, Float64}}()
V_table = Dict{Any, Float64}()
learning_rate = 0.9
explore_rate = 0.7
Vmdp = Qlearning(Q_table, V_table, learning_rate, explore_rate, action_space, typemin(Float64), typemax(Float64))
nb_episode_size = 100
nb_max_episode = 30
nb_sim = 20
Training(Vmdp, nb_episode_size, nb_max_episode, nb_sim, epsilon, particles_b0, pomdp, state_grid_distance)

### 4. Init POMCGS and run the solver

In [ ]:
# --- 1: Define POMCGS and fsc ---
planner = ContinuousPlannerPOMCGS(rmin, Vmdp)
InitPlannerParameters(planner, 
                    nb_process_samples, 
                    max_b_gap,
                    c,
                    discount(pomdp),
                    nb_abstract_obs,
                    true,
                    state_grid_distance,
                    epsilon)
fsc = InitFSC(max_b_gap, planner._max_graph_node_size, action_space)
fsc._map_discrete2continuous_states = init_map_b0_d2c_states
planner._bool_APW = false

In [ ]:
# --- 2: Graph Search ---
Search(pomdp, particles_b0, dict_weighted_b0, fsc, planner, max_planning_secs)

### 5. Simulate the computed FSC

In [ ]:
SimulationFSC(b0, pomdp, fsc, discount(pomdp), planner)

### 6. Evaluate the computed FSC

In [ ]:
nb_eval = 100000
EvaluationWithSimulationFSC(pomdp, fsc, discount(pomdp), nb_eval, planner)